In [0]:
import keras
from keras.datasets import mnist

In [0]:
batch_size = 128
num_classes = 10
epochs = 30

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
from keras import backend

In [0]:
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

In [0]:
x_train = x_train.astype('float')
x_test = x_test.astype('float')
x_train /= 255
x_test /= 255

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
from keras.models import Sequential
model = Sequential()

In [0]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [0]:
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.RMSprop(lr = 0.001, rho=0.9, epsilon=1e-08, decay = 0.0), metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 9s 145us/step - loss: 0.2609 - accuracy: 0.9169 - val_loss: 0.0783 - val_accuracy: 0.9719
Epoch 2/30
60000/60000 [==============================] - 8s 134us/step - loss: 0.0756 - accuracy: 0.9778 - val_loss: 0.0333 - val_accuracy: 0.9895
Epoch 3/30
60000/60000 [==============================] - 8s 132us/step - loss: 0.0561 - accuracy: 0.9831 - val_loss: 0.0314 - val_accuracy: 0.9895
Epoch 4/30
60000/60000 [==============================] - 8s 132us/step - loss: 0.0437 - accuracy: 0.9874 - val_loss: 0.0229 - val_accuracy: 0.9933
Epoch 5/30
60000/60000 [==============================] - 8s 134us/step - loss: 0.0400 - accuracy: 0.9881 - val_loss: 0.0204 - val_accuracy: 0.9921
Epoch 6/30
60000/60000 [==============================] - 8s 134us/step - loss: 0.0356 - accuracy: 0.9893 - val_loss: 0.0250 - val_accuracy: 0.9926
Epoch 7/30
60000/60000 [==============================] - 8s 1

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [0]:
from keras.callbacks import ReduceLROnPlateau
annealer_callback = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)

In [0]:
model.fit_generator(generator=datagen.flow(x_train, y_train, batch_size = batch_size), steps_per_epoch= x_train.shape[0] // batch_size, epochs=epochs, validation_data=(x_test,y_test), verbose = 2,
                    callbacks = [annealer_callback])

Epoch 1/30
 - 22s - loss: 0.2462 - accuracy: 0.9331 - val_loss: 0.0251 - val_accuracy: 0.9934
Epoch 2/30
 - 22s - loss: 0.1769 - accuracy: 0.9524 - val_loss: 0.0509 - val_accuracy: 0.9901
Epoch 3/30
 - 22s - loss: 0.1557 - accuracy: 0.9582 - val_loss: 0.0367 - val_accuracy: 0.9933
Epoch 4/30
 - 22s - loss: 0.1446 - accuracy: 0.9609 - val_loss: 0.0338 - val_accuracy: 0.9934

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/30
 - 22s - loss: 0.1086 - accuracy: 0.9706 - val_loss: 0.0290 - val_accuracy: 0.9935
Epoch 6/30
 - 22s - loss: 0.1098 - accuracy: 0.9707 - val_loss: 0.0336 - val_accuracy: 0.9932
Epoch 7/30
 - 22s - loss: 0.1089 - accuracy: 0.9711 - val_loss: 0.0262 - val_accuracy: 0.9939
Epoch 8/30
 - 22s - loss: 0.1002 - accuracy: 0.9726 - val_loss: 0.0315 - val_accuracy: 0.9932
Epoch 9/30
 - 22s - loss: 0.1036 - accuracy: 0.9720 - val_loss: 0.0384 - val_accuracy: 0.9923
Epoch 10/30
 - 22s - loss: 0.0975 - accuracy: 0.9733 - val_loss: 0.0333 -

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
score

[0.030912711060268692, 0.9937999844551086]